<a href="https://colab.research.google.com/github/sanchit-gandhi/notebooks/blob/main/gemma-transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemma in 🤗 Transformers

Gemma is a family language models released by Google DeepMind in the paper [Gemma: Open Models Based on Gemini Research and Technology](https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf). Gemma leverages the same research and technology used to create the Gemini models. It was trained using a dataset consisting of 6-trillion tokens, formed from web documents, code and mathematics. The result is a series of state-of-the-art models at the 2B and 7B scale, all of which are open-sourced and permissively licensed.

Gemma has support in the 🤗 Transformers library from day-0, in both PyTorch and JAX. In this Google Colab, we'll showcase how to use the Gemma 7B model in PyTorch, leveraging the familiar Transformers 3-line API. By quantising the model to 4-bit precision, the notebook can be run on a Google Colab T4 free-tier GPU. We'll wrap the loaded model into a Gradio demo, showcasing how the model can be shared with anyone in the community.

## Set-Up Environment

First, we need to register our Hugging Face Hub token with our Google Colab runtime. Since the Gemma model is gated, our token will be checked when the model is downloaded to ensure we have accepted the terms-of-use. To register your token, click the key symbol 🔑 in the left-hand pane of the screen. Name the secret `HF_TOKEN`, and copy a token from your Hugging Face [Hub account](https://huggingface.co/settings/tokens). Your token should now be registered, allowing you to access the Gemma weights to this Colab session.

For reasonable inference speed with Gemma, we'll want to run the model on a GPU. The runtime is already configured to use the free 16GB T4 GPU provided through Google Colab Free Tier, so all you need to do is hit `"Connect T4"` in the top right-hand corner of the screen.

Once we've done that, we can go ahead and install the necessary Python packages:

In [ ]:
!pip install --upgrade --quiet transformers accelerate bitsandbytes gradio

## Baseline Implementation

In this example, we'll use Transformers' model + tokenizer API. Let's go ahead an import the two Python classes we'll need to run the Gemma model. The first of these classes, [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForCausalLM), is our model class. This is the Python class that holds the model weights and graph definition. The second class, [`AutoTokenizer`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer), is the tokenizer class, which converts the input string to a token id (discrete number) representation:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

We can now load the pre-trained model weights from the Hugging Face Hub. There are four pre-trained Gemma checkpoints from which we can choose from, summarised in the table below:

| Model ID    | Size / B params | Type        |
|-------------|-----------------|-------------|
| [gemma-2b](https://huggingface.co/google/gemma-2b)    | 2.5             | Base        |
| [gemma-2b-it](https://huggingface.co/google/gemma-2b-it) | 2.5             | Instruction |
| [gemma-7b](https://huggingface.co/google/gemma-7b)    | 8.5             | Base        |
| [gemma-7b-it](https://huggingface.co/google/gemma-7b-it) | 8.5             | Instruction |


For this example, we'll use [gemma-7b-it](https://huggingface.co/google/gemma-7b-it), the 8.5B parameter model that has been instructioned tuned for the task of an assistant (or chatbot). An 8.5B parameter LLM in full precision (float32) requires 16GB of memory just to load the weights. The GPU typically assigned to a Google Colab free tier only has a capacity of 16GB. This means we are at risk of an *out-of-memory (OOM)* error, where the memory of the model exceeds that of the GPU. To circumvent this, we'll load the weights in [4-bit precision](https://huggingface.co/blog/4bit-transformers-bitsandbytes), which reduces the memory of the weights roughly by a factor of 8. This is extremely simple in Transformers: we just pass the flag [`load_in_4bit`](https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained.load_in_4bit) when we load the pre-trained weights:

In [ ]:
checkpoint_id = "google/gemma-7b-it"

model = AutoModelForCausalLM.from_pretrained(checkpoint_id, low_cpu_mem_usage=True, load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_id)

The advantage of using the `AutoModelForCausalLM` and `AutoTokenizer` API over model-specific classes is that we can easily swap the checkpoint id for any other checkpoint
on the Hugging Face Hub and re-use our code without any changes. The auto-classes will take care of loading the correct model and tokenizer classes for us! That means if a new LLM is released, we can quickly update our code to leverage the new model.

Great! We've loaded our model into memory, so now we're ready to define our pre-processing strategy. Our inputs consist of a series of user/assistant responses, from which we want to generate a new assistant response. Each instruction tuned model has it's own format for user/assistant responses, and it can be fiddly getting one's inputs into the correct format for the model. Luckily for us, the tokenizer provides a convinient method [`apply_chat_template`](https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.apply_chat_template) to format the tokens correctly for us. We just need to define what messages were sent by the user, and what ones were provided by the assistant. The tokenizer will then handle the precise formatting, and getting the ids ready for the model:

In [ ]:
messages = [
    {"role": "user", "content": "Can you tell me about current trends artificial intelligence?"},
    {"role": "assistant", "content": "Certainly! Artificial Intelligence (AI) is advancing rapidly, with key trends including the growing adoption of deep learning techniques, the increasing use of AI in healthcare, finance, retail, and other industries, and the development of new AI tools for creative tasks, like music composition and visual art creation. Additionally, there is a growing focus on ethical considerations surrounding AI, as concerns arise about potential bias, privacy implications, and job displacement. Furthermore, AI is integrating with other technologies, such as blockchain and virtual reality, to create novel solutions for various challenges."},
    {"role": "user", "content": "What should I consider when using large language models?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

Perfect! Now, our model is currently sitting on the GPU:

In [ ]:
model.device

Whereas our inputs are on the CPU:

In [ ]:
encodeds.device

Every PyTorch model expects the inputs to be on the same device as the model. Let's move the inputs to the correct device before generating:

In [ ]:
model_inputs = encodeds.to(model.device)

Now that we've pre-processed our inputs, we can generate our response using the model in much the same way as before:

In [ ]:
generated_ids = model.generate(model_inputs, max_new_tokens=1024, do_sample=True)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_text[0])

Looks like a very reasonable response! We can see the change between the user input and the assistant response marked by the `[INST]` and `[/INST]` tokens. This was handled automatically for us by the tokenizer. Note that we've generated with [*sampling*](https://huggingface.co/blog/how-to-generate#sampling), so there's an element of randomness in the generations, meaining each one will be different from the last. Feel free to re-run generation a few times to get a feel for the kind of variety that sampling brings.

> **Note:** the responses generated by the model are purely for demonstration purposes. They should not be taken as financial recommendations.

## Streaming Outputs

While the generated response looks good, we had to wait a significant amount of time for the model to finish generating. We know from our knowledge of Transformer models that they generate in an *auto-regressive* fashion, that is, one token at a time:

<figure class="image table text-center m-0 w-full">
    <video
        style="max-width: 90%; margin: auto;"
        autoplay loop muted playsinline
        src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/assisted-generation/gif_2_1080p.mov"
    ></video>
</figure>

Rather than waiting for the entire text sequence to finish generating, we can print each token as it is generated. While this doesn't change the overall latency of the model (the total generation time is the same as before), the *percieved* latency is lower, since the first generated token is returned to the user as soon as it's ready.

Transformers provides a [`TextStreamer`](https://huggingface.co/docs/transformers/internal/generation_utils#transformers.TextStreamer) class to easily print the predictions on-the-fly. We simply have to pass the streamer to our generate function, and the rest is taken care for us. Let's run an example using the text streamer:

In [ ]:
from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
generated_ids = model.generate(model_inputs, max_new_tokens=1024, do_sample=True, streamer=streamer)

Interacting with the LLM this way already feels more personalised, as we watch it generate each token at a time. It also feels faster to the user, as we get the first output as soon as it is ready.

## Gradio Demo

While it's interesting for us learners to interact with the model through Python code, it's by no means the easiest way to share our model with others. To finish this section, we'll demonstrate how the Gemma model can be wrapped into a simple chat interface and shared with anyone in the community.

To achieve this, we'll use the [Gradio library](https://www.gradio.app). Gradio is an open-source Python library that simplifies the creation of web interfaces for machine learning models. It allows developers and researchers to quickly build and share customisable UI components for model testing and feedback, enhancing the accessibility and usability of AI models.

First, we'll define an end-to-end function that takes the user's message and the chat history, and generates a response. Largely speaking, this simply requires concatenating the three stages of prediction:
1. Tokenizer (pre-process) the message and chat history
2. Define the generation arguments
3. Generate the predicted ids and stream the output

We'll make a small modification to our streamer: instead of using the base [`TextStreamer`](https://huggingface.co/docs/transformers/internal/generation_utils#transformers.TextStreamer), we'll use the [`TextIteratorStreamer`](https://huggingface.co/docs/transformers/internal/generation_utils#transformers.TextIteratorStreamer), which returns the generated ids in a non-blocking way.

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread

MAX_INPUT_TOKEN_LENGTH = 4096

def generate(message, chat_history):
    # Step 1: pre-process the inputs
    conversation = []
    for user, assistant in chat_history:
        conversation.extend([{"role": "user", "content": user}, {"role": "assistant", "content": assistant}])

    conversation.append({"role": "user", "content": message})

    input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt")

    # in-case our inputs exceed the maximum length, we might need to cut them
    if input_ids.shape[1] > MAX_INPUT_TOKEN_LENGTH:
        input_ids = input_ids[:, -MAX_INPUT_TOKEN_LENGTH:]
        gr.Warning(f"Trimmed input from conversation as it was longer than {MAX_INPUT_TOKEN_LENGTH} tokens.")

    input_ids = input_ids.to(model.device)
    streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

    # Step 2: define generation arguments
    generate_kwargs = dict(
        {"input_ids": input_ids},
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
    )

    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    # Step 3: generate and stream outputs
    outputs = ""
    for text in streamer:
        outputs += text
        yield outputs

With the generation function defined, we can now create a Gradio demo in just 3 lines of additional code:

In [ ]:
import gradio as gr

chat_interface = gr.ChatInterface(generate)
chat_interface.queue().launch(share=True)